### Generación de los informes Consello Orientador

#### Funciones útiles

In [90]:
def resuelve(vals):
    '''
    Devuelve un valor representativo del global
    de las calificaciones de los docentes de esta competencia clave
    
    En este caso se devuelve el promedio de las calificaciones en una escala 0-3'''
    pos=range(len(vals))
    if sum(vals):
        #return round(1.0*sum([u*v for u,v in zip(vals,pos)])/len(vals))
        return 1.0*sum(vals)/len(vals)
    return 0

def nomtohtml(n):
    '''
    converts utf names to html names
    '''
    from re import sub
    
    acute=[
        ['á','a'],['Á','A'],['é','e'],['É','E'],['í','i'],['Í','I'],['ó','o'],['Ó','O'],['ú','u'],['Ú','U']
    ]
    tilde=[
        ['ñ','n'],['Ñ','N']
    ]
    #print n
    for item in acute:
        n=sub(item[0],'&'+item[1]+'acute;',n)
    #print n
    for item in tilde:
        n=sub(item[0],'&'+item[1]+'tilde;',n)
    #print n
    return n

def informe_competencias(curso,nome,resp,dia,mes,tratam,tutor,nombre):
    '''
    Genera el Consello orientador del alumno nome, del curso curso
    cuyo tutor recibe el tratamiento tratam (Tutor/Tutora), de nombre tutor que se 
    guardará en el fichro html de nombre nombre
    '''
    
    fich=open(PATH+'/plantilla_informe.html')
    aux=fich.readlines()
    fich.close()
    base=''.join(aux)
    try:
        fich=open(PATH+'/plantilla_orientac'+curso[0]+'eso.html')
    except:
        print('No encuentro la plantilla',PATH+'/plantilla_orientac'+curso[0]+'eso.html')
        return
    aux=fich.readlines()
    orientaeso=''.join(aux)
   
    relleno=[]
    relleno.append(nome)
    relleno.append(nome)
    relleno+=resp
    relleno.append(orientaeso)
    relleno.append(dia)
    relleno.append(nomtohtml(mes))
    relleno.append(tratam)
    relleno.append(tutor)
    
    fich=open(nombre+'.html','w')
    fich.write(base % tuple(relleno))
    fich.close()

In [82]:
import os
from AVBD import AV_BD



#Niveles recomendados por la Consellería para el Consello Orientador del curso 2016-17
label=['Insuficiente', 'Suficiente', 'Ben', 'Notable' , 'Sobresaliente']
lobs=['Non acadou os obxectivos',
      'Acadou os obxectivos suficientemente',
      'Acadou os obxectivos de xeito destacado']

avbd=AV_BD('Path-to-actual-AVBD')
PATH='Path-to-templates-directory'
PATHINF='Path-to-root-of-outputs'
aval=3 #Evaluación a consioderar

tutores=avbd.busca('SELECT Aulas.nome, Rol.nome, Rol.apelidos FROM Rol join Aulas join Listados WHERE Rol.id=Listados.RolId AND Aulas.id=Listados.AulasId AND Listados.rol="tutor"' )
contador=0
competencias=avbd.busca('SELECT id, nome, abrev FROM ListaComp')
for tutor in tutores:
    curso, nomT, apT= tutor
    if not 'ESO' in curso:
        continue
    
    #Seleccion del tratamiento: tutor/tutora
    if nomT.split()[0] in [u'nom1', u'nom2',....]:
        tratam='O titor'
        nometitor=nomtohtml('D. '+nomT.encode('utf8')+' '+apT.encode('utf8'))
    else:
        tratam='A titora'
        nometitor=nomtohtml('D.&ordf; '+nomT.encode('utf8')+' '+apT.encode('utf8'))
    
    #Selecciona la fecha de la evaluacion
    if curso.startswith('4'):
        dia=27
    else:
        dia=26
    mes='Xuño'
    
    #Crea un directorio para los CO de la tutoría en el directorio root
    cad='mkdir %s' % PATHINF+'/'+curso
    os.system(cad)
    
    
    for alum in avbd.extraeGrupo(grupo=curso):
       
        RolId=avbd.buscaRolId(alum[1],alum[2])[0][0]
        
        wc=[]
        res=[]
        #Selecciona las opciones que deben aparecer rellenas en el informe
        for item in competencias:
            contador+=1
            vals=avbd.busca('SELECT valor FROM Competencias WHERE RolId=%i AND  ListaCompId=%i AND aval="%s"' % (RolId,item[0],aval))
            vals=[int(i[0]) for i in vals]
            if vals:
                if item[0]==1:
                    wc.append(int(round(2.0*(1.0-resuelve(vals)/3))))
                    aux=[' ']*3
                    aux[wc[-1]]='selected'
                    res+=aux
                else:
                    wc.append(int(round(4.0*(1.0-resuelve(vals)/3))))
                    aux=[' ']*6
                    aux[wc[-1]+1]='selected'
                    res+=aux
               
            else:
                #Sin valores
                wc.append(-1)
                aux=[' ']*6
                res+=aux
        nomalum=nomtohtml(alum[2].encode('utf8')+', '+alum[1].encode('utf8'))
        informe_competencias(curso,nomalum,res,dia,mes,tratam,nometitor,PATHINF+'/'+curso+'/'+alum[2]+', '+alum[1])